In [3]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas
from scipy.optimize import curve_fit 
from numpy import log, exp, linspace, sqrt, diag
from sklearn.linear_model import LinearRegression

df = pandas.read_csv( 'data/assay_data_clean.csv' )
temp_key = pandas.read_csv( 'data/temp_key.csv' )
temp_key.index = temp_key.Well
df['temp'] = df.well.str[0].map( temp_key.Kelvin.to_dict() ) 
df.dropna( inplace=True )

# logistic equation 
def f(x, x0, k): 
    return 1/(1+exp(-k*(x-x0)))

# util function 
def fit_and_make_plot( df ):
    name = df.mutant.unique()[0]
    df.rate = df.rate / df.rate.max()
    
    # linear fit 
    reg = LinearRegression()
    reg.fit( df.temp.reshape(-1, 1), df.rate )
    slope = reg.coef_[0]
    
    # try fitting to logistic eqn 
    try:
        p0 = ( df.temp.mean(), slope )
        popt, pcov = curve_fit( f, df.temp, df.rate, p0=p0 )
        perr = sqrt( diag( pcov ) )
        return pandas.Series( dict( zip( ( 'tm', 'k' ) , popt ) ) ) 
    except Exception as e:
        print e
        return pandas.Series()        
    
    # plot
    plt.figure( figsize=(2,2) )
    plt.scatter( df.temp, df.rate, alpha=0.7, color='black', marker='.' )
    x_space = linspace( df.temp.min(), df.temp.max(), 100 )
    plt.plot( x_space, f( x_space, *popt ), alpha=0.5, color='purple' )
    plt.xlabel( 'T $(K)$' )
    plt.ylabel( '$min^{-1}$')
    plt.xticks( [ 303, 313, 323 ] )
    plt.yticks( [ 0, 0.25, 0.5, 0.75, 1 ] )
    plt.title( df.mutant.unique()[0] )
    plt.savefig( 'plots/%s.png' % name )
    plt.close()
    
fits = df.groupby( by='mutant' ).apply( fit_and_make_plot )
fits.sample(10)

,k,tm
mutant,,
W120A,-6.690186,311.895265
W325L,-1.320812,315.765132
E177A,-0.487548,310.457311
H122N,-0.959199,-177.717952
Y295A,0.351082,325.548702
W120F,-0.462998,314.660527
W328C,-0.898275,312.330769
Q316R,-0.591794,313.257592
W399C,-0.008997,412.749503


In [2]:
fits['native'] = fits.index.str[0]
fits['designed'] = fits.index.str[-1]
fits['position'] = fits.index.str[1:-1].astype( int )
fits['mutant'] = fits.index
fits.head()

,k,tm,native,designed,position,mutant
mutant,,,,,,
A195S,-0.906300,312.232598,A,S,195,A195S
A356A,-1.124698,313.038266,A,A,356,A356A
A357A,-0.325209,312.244581,A,A,357,A357A
A408A,-1.260728,313.105931,A,A,408,A408A
C167Q,-0.532181,311.621618,C,Q,167,C167Q
